In [1]:
import kfp
from kfp.components import load_component_from_url
import datetime

# Settings

In [ ]:
n_parallel_writes = 10
filesize_per_write = "200M"  # (in fallocate -l format, eg 200M, 3G, ...)

# Define components and pipeline

In [2]:
create_junk = load_component_from_url("https://raw.githubusercontent.com/ca-scribner/kubeflow-pipelines-components/master/components/files/create_junk/component.yaml")
put_to_minio = load_component_from_url("https://raw.githubusercontent.com/ca-scribner/kubeflow-pipelines-components/master/components/minio/put_to_minio/component.yaml")

In [3]:
@kfp.components.create_component_from_func
def range_component(start: int=0, stop: int=None, step: int=1) -> list:
    """Operates like Python's range, but in kfp as a component"""
    if stop is None:
        raise ValueError(f"Invalid value '{stop}' for stop")
    return list(range(int(start), int(stop), step))

In [4]:
def get_current_namespace():
    import kubernetes
    kubernetes.config.load_incluster_config()
    kube_client = kubernetes.client.CoreV1Api()
    current_namespace = open("/var/run/secrets/kubernetes.io/serviceaccount/namespace").read()
    return current_namespace
namespace = get_current_namespace()

In [5]:
def pipeline(n_writes):
    data = create_junk(size="200M")

    iterations = range_component(stop=n_writes)

    with kfp.dsl.ParallelFor(iterations.output) as iteration:
        put_to_minio(
            source=data.output,
            target=f'{namespace}/debug_minio/{kfp.dsl.RUN_ID_PLACEHOLDER}/{iteration}',
            tenant='standard',
        )

# Run pipeline

In [6]:
c = kfp.Client()

In [7]:
now = datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
c.create_run_from_pipeline_func(
    pipeline,
    arguments={'n_writes': 50},
    run_name=f'MinIO Parallel Writes Test ({namespace}/{now})',
    experiment_name=f'MinIO Parallel Writes Test',
    namespace=namespace
)

RunPipelineResult(run_id=bf0969e2-0ead-4c01-9b08-2e05d219a2d1)